In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L3dvcmtzcGFjZXMvY29uanVudHVyYV9yZWxhdG9yaW8='
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
# globals()["__spec__"] = None

{"/opt/conda/lib/python3.12/importlib/_bootstrap.py": 1718724512.0, "/opt/conda/lib/python3.12/importlib/_bootstrap_external.py": 1718724512.0, "/opt/conda/lib/python3.12/zipimport.py": 1718724511.0, "/opt/conda/lib/python3.12/codecs.py": 1718724511.0, "/opt/conda/lib/python3.12/encodings/aliases.py": 1718724511.0, "/opt/conda/lib/python3.12/encodings/__init__.py": 1718724511.0, "/opt/conda/lib/python3.12/encodings/utf_8.py": 1718724511.0, "/opt/conda/lib/python3.12/abc.py": 1718724511.0, "/opt/conda/lib/python3.12/io.py": 1718724511.0, "/opt/conda/lib/python3.12/stat.py": 1718724511.0, "/opt/conda/lib/python3.12/_collections_abc.py": 1718724511.0, "/opt/conda/lib/python3.12/genericpath.py": 1718724511.0, "/opt/conda/lib/python3.12/posixpath.py": 1718724511.0, "/opt/conda/lib/python3.12/os.py": 1718724511.0, "/opt/conda/lib/python3.12/_sitebuiltins.py": 1718724511.0, "/opt/conda/lib/python3.12/encodings/utf_8_sig.py": 1718724511.0, "/opt/conda/lib/python3.12/site-packages/_distutils_ha

/tmp/ipykernel_62058/4181915731.py:25: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



In [2]:
# Bibliotecas
import pandas as pd

# Coleta de dados
# Sidra/IBGE
# Escolha as variáveis e, a seguir, clique no botão "Links de Compartilhar" e selecione "Parâmetros para a API"
dados_sidra = pd.read_json(
"https://apisidra.ibge.gov.br/values/t/7060/n1/all/v/63,2265/p/all/c315/7169,7170,7445,7486,7558,7625,7660,7712,7766,7786/d/v63%202,v2265%202?formato=json"
)

# BCB
# No SGS/BCB, procure pelo código da série (aqui, '13521' é o da meta de inflação)
# No Google, após procurar 'api sgs bcb', colete a URL e então modifique o parâmetro no link
dados_bcb = pd.read_json(
"https://api.bcb.gov.br/dados/serie/bcdata.sgs.13521/dados?formato=json"
)

# Tratamento dos dados
dados_ipca = (
    dados_sidra
    .rename(
        columns = {
            "V": "valor",
            "D2N": "variacao",
            "D3C": "data",
            "D4N": "grupo"
        }
    )
    .query("valor not in ['Valor', '...']")
    .assign(
        data = lambda x: pd.to_datetime(x.data, format= "%Y%m"),
        variacao = lambda x: x.variacao.case_when([
            (x.variacao == "IPCA - Variação mensal", "Var. % mensal"),
            (x.variacao == "IPCA - Variação acumulada em 12 meses", "Var. % acum. 12m")
        ]),
        grupo = lambda x: x.grupo.str.replace(pat = "^\\d{1}\\.", repl = "", regex = True),
        valor = lambda x: x.valor.astype(float)
    )
    .filter(["data", "variacao", "grupo", "valor"])
)

dados_meta = dados_bcb.assign(
    data = lambda x: pd.to_datetime(x.data, format = "%d/%m/%Y")
    ).rename(columns = {"valor": "Meta % anual"}).set_index("data")

# Vamos criar duas tabelas
tabela_g1 = (
    dados_ipca
    .query("grupo == 'Índice geral'") # Selecionando apenas o IPCA cheio
    .drop(labels = "grupo", axis = "columns") # Retirando a coluna de grupos
    .pivot(index = "data", columns = "variacao", values = "valor") # Desimpilhando para cruzar com os dados_meta
    .join(other = dados_meta, how = "left")
    .assign(**{"Meta % anual": lambda x: x["Meta % anual"].ffill()})
    .reset_index()
    .melt(id_vars = "data", var_name = "variavel", value_name = "valor") # Empilhando novamente
)

# Tabela com a variação no último mês dos grupos do IPCA
tabela_g2 = (
    dados_ipca
    .query("grupo != 'Índice geral' and variacao == 'Var. % mensal' and data == data.max()")
)

variacao = tabela_g1.query("data == data.max() and variavel == 'Var. % mensal'").valor.iloc[0]
periodo = tabela_g1.query("data == data.max() and variavel == 'Var. % mensal'").data.iloc[0].strftime("%B/%Y")